In [1]:
import sys
sys.path.append('../Metaheuristic Algorithms/') 
import numpy as np
import tensorflow as tf
from tensorflow.keras.datasets import cifar10
from tensorflow.keras.utils import to_categorical
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Conv2D, MaxPooling2D, Dropout, Flatten, Dense
from DESSA import DESSA  # Importing your DESSA class from the DESSA.py file

In [2]:
# Load CIFAR-10 data
(x_train, y_train), (x_test, y_test) = cifar10.load_data()

# Normalize pixel values to be between 0 and 1
x_train, x_test = x_train / 255.0, x_test / 255.0

# One-hot encode the labels
y_train, y_test = to_categorical(y_train), to_categorical(y_test)

In [3]:
def create_cnn_model(params, input_shape=(32, 32, 3), num_classes=10):
    num_filters, filter_size, dropout_rate = int(params[0]), int(params[1]), params[2]
    model = Sequential([
        Conv2D(num_filters, (filter_size, filter_size), padding='same', activation='relu', input_shape=input_shape),
        MaxPooling2D(pool_size=(2, 2)),
        Dropout(dropout_rate),
        Flatten(),
        Dense(256, activation='relu'),
        Dropout(0.5),
        Dense(num_classes, activation='softmax')
    ])
    model.compile(optimizer='adam', loss='categorical_crossentropy', metrics=['accuracy'])
    return model

In [4]:
def objective_function(params):
    model = create_cnn_model(params)
    history = model.fit(x_train, y_train, epochs=3, batch_size=64, validation_split=0.1, verbose=0)
    return -max(history.history['val_accuracy'])  # Negative accuracy for minimization

In [5]:
# Define bounds for DESSA parameters: number of filters, filter size, and dropout rate
bounds = [(16, 64), (2, 5), (0.1, 0.5)]

# Initialize DESSA with the objective function, bounds, population size, and number of iterations
dessa_optimizer = DESSA(objective_function, bounds, population_size=10, iterations=10)

# Perform optimization
best_params, best_fitness, fitness_history = dessa_optimizer.optimize()

# Output the best parameters and the corresponding validation accuracy
print("Best Parameters:", best_params)
print("Best Validation Accuracy:", -best_fitness)  # Convert fitness back to positive for accuracy

Best Parameters: [48.9914755   4.17604678  0.36770232]
Best Validation Accuracy: 0.6326000094413757


In [6]:
final_model = create_cnn_model(best_params, input_shape=(32, 32, 3), num_classes=10)
final_history = final_model.fit(x_train, y_train, epochs=20, validation_data=(x_test, y_test), verbose=1)
print("Final model accuracy on test set:", final_history.history['val_accuracy'][-1])

Epoch 1/20
1563/1563 [==============================] - 17s 11ms/step - loss: 1.6319 - accuracy: 0.4081 - val_loss: 1.3404 - val_accuracy: 0.5264
Epoch 2/20
1563/1563 [==============================] - 16s 10ms/step - loss: 1.3790 - accuracy: 0.5049 - val_loss: 1.1905 - val_accuracy: 0.5815
Epoch 3/20
1563/1563 [==============================] - 15s 10ms/step - loss: 1.2784 - accuracy: 0.5444 - val_loss: 1.1752 - val_accuracy: 0.5837
Epoch 4/20
1563/1563 [==============================] - 15s 10ms/step - loss: 1.2116 - accuracy: 0.5670 - val_loss: 1.1270 - val_accuracy: 0.6014
Epoch 5/20
1563/1563 [==============================] - 15s 10ms/step - loss: 1.1609 - accuracy: 0.5887 - val_loss: 1.0786 - val_accuracy: 0.6202
Epoch 6/20
1563/1563 [==============================] - 15s 10ms/step - loss: 1.1170 - accuracy: 0.6056 - val_loss: 1.0661 - val_accuracy: 0.6258
Epoch 7/20
1563/1563 [==============================] - 15s 10ms/step - loss: 1.0785 - accuracy: 0.6147 - val_loss: 1.0141 -